In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

# Preprocess Data
In the <b>HW3_template</b> folder you will find `TSLA.csv`, `GOOGL.csv` and `DJI.csv` files. Use Pandas (You have used it in HW1) to retrieve the dataset. Use only <b>Open</b> price as your input. (You will train three models for three different stocks, don't mix these data together!)

In [4]:
data_TSLA = pd.read_csv("TSLA.csv") 
data_GOOGL = pd.read_csv("GOOGL.csv")
daat_DJI = pd.read_csv("DJI.csv")
data_TSLA = data_TSLA.sort_values('Date')

print(data_TSLA.head())
[n, _] = data_TSLA.shape


         Date       Open   High        Low      Close  Adj Close    Volume
0  2010-06-29  19.000000  25.00  17.540001  23.889999  23.889999  18766300
1  2010-06-30  25.790001  30.42  23.299999  23.830000  23.830000  17187100
2  2010-07-01  25.000000  25.92  20.270000  21.959999  21.959999   8218800
3  2010-07-02  23.000000  23.10  18.709999  19.200001  19.200001   5139800
4  2010-07-06  20.000000  20.00  15.830000  16.110001  16.110001   6866900


## Normalize your data
You could use `MinMaxScaler` in `sklearn.preprocessing` to normalize the data between 0 and 1.

In [5]:
def preprocessData(data, num_seq=20):
    scaler = MinMaxScaler(feature_range=(0, 1))
    open_scaled =  scaler.fit_transform(data_TSLA['Open'].values.reshape(-1,1))
#     data_processed = pd.DataFrame(index=range(0,len(data)),columns=['Date', 'Open'])
#     data_processed['Date'] = data['Date']
#     data_processed['Open'] = data['Open']
 
    n =len(data)
    x, y = [], []
    
    for i in range(num_seq, n):
        x.append(open_scaled[i-num_seq:i, 0])
        y.append(open_scaled[i, 0])
    x, y = np.array(x), np.array(y)
    x = np.reshape(x, (-1, num_seq, 1))
    return x,y

# Split training, validation and testing data
<p style="font-size:20px">Since you will impelement a many-to-one Recurrent Neural Network model, every input data will have shape [batch_size, num_seq, input_size] and output data will have shape [batch_size, input_size] 

In [7]:
num_seq = 20;
[x, y] = preprocessData(data_TSLA)
N = len(x)
test_split = int(N*.90)
val_split = int(N*.75)
train_x, train_y = x[:val_split], y[:val_split]
val_x, val_y = x[val_split:test_split], y[val_split:test_split]
test_x, test_y = x[test_split:], y[test_split:]
train_dates, val_dates, test_dates = data_TSLA[:val_split], data_TSLA[val_split:test_split], data_TSLA[test_split:]

# TesnorFlow Model

In [33]:
input_size=1
lstm_size=128
num_layers=1
keep_prob=0.8
batch_size = 128
learning_rate = 0.001
init_epoch = 5
num_epoch = 1000

tf.reset_default_graph()
lstm_graph = tf.Graph()

X = tf.placeholder(tf.float32, [None, num_seq, 1], name = 'X')
Y = tf.placeholder(tf.float32, [None])

with tf.name_scope('input'):
    x = tf.unstack(X, num_seq, axis = 1)

with tf.name_scope('rnn'):
    # Define a lstm cell
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0)
    # Get LSTM cell outputs and states
    rnn_outputs, rnn_states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    # For tensorboard
    
with tf.name_scope('nn'):
    # Define the first ANN
    nn_outputs = tf.layers.dense(rnn_outputs[-1], 64 , activation=tf.nn.tanh, name = 'nn1')
    # Define the second ANN
    prediction = tf.layers.dense(nn_outputs, 1, name = 'nn2')

loss = tf.reduce_mean(tf.square(prediction - Y), name = 'loss')
optimizer = tf.train.AdamOptimizer(learning_rate, name = 'optimizer').minimize(loss)


# Training

In [35]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epoch):
        for start in range(0, len(train_x) + 1, batch_size):
            # Create training batches
            batch_x = train_x[start:min(start+batch_size, len(train_x))]
            batch_y = train_y[start:min(start+batch_size, len(train_y))]

            # Run optimization operation (backprop)
            sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})
            
        if (i+1)%100 == 0:
            acc = sess.run(loss,feed_dict={X:val_x, Y:val_y})
            print("trained "+str(i+1)+" vadliation accuracy" + str(acc))
    print("Final Validation Accuracy:", sess.run(loss, feed_dict={X:val_x, Y:val_y}))
             

trained 100 vadliation accuracy0.0199089
trained 200 vadliation accuracy0.0502842
trained 300 vadliation accuracy0.0775987
trained 400 vadliation accuracy0.0530007
trained 500 vadliation accuracy0.0580432
trained 600 vadliation accuracy0.06533
trained 700 vadliation accuracy0.0631326
trained 800 vadliation accuracy0.0670445
trained 900 vadliation accuracy0.0650421
trained 1000 vadliation accuracy0.0644573
Final Validation Accuracy: 0.0644573


# Testing